In [20]:
%load_ext autoreload
%autoreload 2
from mtsccleav import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeClassifierCV

In [22]:
from aeon.classification.convolution_based import (
    MiniRocketClassifier, MultiRocketClassifier, HydraClassifier,
)

In [23]:
from aeon.utils.conversion import convert_collection
from aeon.utils.validation import get_type
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from aeon.transformations.collection.convolution_based import Rocket
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


In [24]:
from aeon.classification.convolution_based import (
    Arsenal,
    HydraClassifier,
    MiniRocketClassifier,
    MultiRocketClassifier,
    MultiRocketHydraClassifier,
    RocketClassifier,
)

In [25]:
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score,
    confusion_matrix, matthews_corrcoef
)
import numpy as np

In [26]:
from sklearn.model_selection import StratifiedKFold

In [27]:
ts_panel = load_as_pd_multiindex("../data/01_cum_st.csv")
X = construct_X(ts_panel, include_five_p_cleav=False, include_prob=False, include_ss=False)


In [28]:
X

three_p_cleav_1  three_p_cleav_compl_1
instances time_points                                        
0         0                          2                     -8
          1                          3                     -9
          2                          5                     -9
          3                          6                    -10
          4                          8                    -12
...                                ...                    ...
1653      10                        -4                      1
          11                        -2                      1
          12                        -1                      0
          13                         0                     -1
          14                        -2                      1

[24810 rows x 2 columns]

In [29]:
X.isnull().values.any()

np.False_

In [30]:
X.fillna(-2, inplace=True)

In [31]:
X = pad_multiindex_ts(X,-2)

In [32]:

X_convert=convert_collection(X, "np-list")

In [33]:
get_type(X), get_type(X_convert)

('pd-multiindex', 'np-list')

In [34]:
y = np.concatenate((np.full((1, 827), 'cleav'), np.full((1, 827), 'non_cleav')), axis=None)
labelencoder = LabelEncoder()
y_transform = labelencoder.fit_transform(y)

In [35]:
def multiclass_specificity(cm):
    FP = cm.sum(axis=0) - np.diag(cm)
    TN = cm.sum() - (FP + cm.sum(axis=1) - np.diag(cm) + np.diag(cm))
    return np.mean(TN / (TN + FP))

In [ ]:
def evaluate_classifier(classifier, X, y, n_splits=5):
    print(f"\n Evaluating {classifier.__class__.__name__}")

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    results = {k: [] for k in ["accuracy", "sensitivity", "specificity", "f1", "mcc"]}

    for fold, (train_idx, test_idx) in enumerate(cv.split(X, y), 1):
        # print(f"Fold {fold}")

        X_train = [X[i] for i in train_idx]
        X_test = [X[i] for i in test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]
    
        # print(y_train)
        # transformer.fit(X_train)
        # X_train_trans = transformer.transform(X_train)
        # X_test_trans = transformer.transform(X_test)

        # clf = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
        # clf.fit(X_train_trans, y_train)
        # y_pred = clf.predict(X_test_trans)

        clf = classifier
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        
        # acc = accuracy_score(y_test, y_pred)
        # sens = recall_score(y_test, y_pred, average="macro")
        # f1 = f1_score(y_test, y_pred, average="macro")
        # mcc = matthews_corrcoef(y_test, y_pred)
        # cm = confusion_matrix(y_test, y_pred)
        # spec = multiclass_specificity(cm)

        # results["accuracy"].append(acc)
        # results["sensitivity"].append(sens)
        # results["specificity"].append(spec)
        # results["f1"].append(f1)
        # results["mcc"].append(mcc)

        sens = recall_score(y_test, y_pred, average="macro")
        # print(sens)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        specificity = tn / (tn+fp)
        sensitivity = tp / (tp+fn)
        f1_score = 2 * tp / (2 * tp + fp + fn)
        mcc = matthews_corrcoef(y_test, y_pred)

        # print(accuracy, specificity, sensitivity, f1_score, mcc)

        results["accuracy"].append(accuracy)
        results["sensitivity"].append(sensitivity)
        results["specificity"].append(specificity)
        results["f1"].append(f1_score)
        results["mcc"].append(mcc)

    print("\nAverage 5-Fold Results:")
    for metric, scores in results.items():
        print(f"{metric.capitalize():<12}: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

In [37]:
evaluate_classifier(RocketClassifier(random_state=18), X_convert, y_transform)
# evaluate_classifier(MiniRocketClassifier(random_state=18), X_convert, y_transform)
# evaluate_classifier(MultiRocketClassifier(random_state=18), X_convert, y_transform)
# evaluate_classifier(HydraClassifier(random_state=18), X_convert, y_transform)
# evaluate_classifier(MultiRocketHydraClassifier(random_state=18), X_convert, y_transform)


🔍 Evaluating RocketClassifier

Average 5-Fold Results:
Accuracy    : 0.7872 ± 0.0112
Sensitivity : 0.8162 ± 0.0247
Specificity : 0.7582 ± 0.0236
F1          : 0.7931 ± 0.0120
Mcc         : 0.5759 ± 0.0227


In [38]:
# 01_single_multi_eq


In [ ]:
import datetime
print(f"This Notebook last end-to-end runs on: {datetime.datetime.now()}\n")